### Обучение TextToLabel

In [267]:
import pandas as pd

files: list[str] = [
    "dataset/transcription/hr_bot_clear.csv",
    "dataset/transcription/hr_bot_noise.csv",
    "dataset/transcription/hr_bot_synt.csv",
    "dataset/transcription/luga.csv",
]

li = []
for filename in files:
    df = pd.read_csv(filename)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df = df.sample(frac=1)
df = df[df['attribute'] != -1]
df.head()

,audio_filepath,id,text,label,attribute,denoise_filepath,transcription
4150,dataset/hr_bot_synt/42647498-76ff-11ee-81a5-c0...,42647498-76ff-11ee-81a5-c09bf4619c03_2,протянуть на один вагон,10,1,dataset/hr_bot_synt_denoise/42647498-76ff-11ee...,проти ночт на один вотон
3187,dataset/hr_bot_synt/3556949b-76ff-11ee-8098-c0...,3556949b-76ff-11ee-8098-c09bf4619c03_1,протянуть на четыре вагона,10,4,dataset/hr_bot_synt_denoise/3556949b-76ff-11ee...,потянуть а четыре вогона
1883,dataset/hr_bot_noise/53f9a06b-76fe-11ee-baf6-c...,53f9a06b-76fe-11ee-baf6-c09bf4619c03,протянуть на пять вагонов,10,5,dataset/hr_bot_noise_denoise/53f9a06b-76fe-11e...,протянуть на ядь вагонов
4223,dataset/hr_bot_synt/7213da20-76ff-11ee-a5b0-c0...,7213da20-76ff-11ee-a5b0-c09bf4619c03_1,протянуть на восемь вагонов,10,8,dataset/hr_bot_synt_denoise/7213da20-76ff-11ee...,протянуть на посу богунув
5547,dataset/hr_bot_synt/6b8e001c-76ff-11ee-8b28-c0...,6b8e001c-76ff-11ee-8b28-c09bf4619c03_1,осадить на одиннадцать вагонов,4,11,dataset/hr_bot_synt_denoise/6b8e001c-76ff-11ee...,осладить к нам они на зо свобоном


In [268]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    HashingVectorizer,
    CountVectorizer,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC


# vectorizer = HashingVectorizer(
#     lowercase=True, n_features=2**10, stop_words="english"
# )
# vectorizer = TfidfVectorizer(
#     lowercase=True, sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english", ngram_range=(1, 2)
# )
vectorizer = CountVectorizer(ngram_range=(1, 2))

# classifier = RandomForestClassifier(n_estimators=200, random_state=42)
# classifier = LogisticRegression()
# classifier = KNeighborsClassifier(n_neighbors=5)
# classifier = MultinomialNB()
classifier = LinearSVC()

pipeline = Pipeline(
    [
        ("tfidf", vectorizer),
        ("classifier", classifier),
    ]
)

In [269]:
X = df["transcription"]
y = df["attribute"]

In [270]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', CountVectorizer(ngram_range=(1, 2))),
                ('classifier', LinearSVC())])

In [271]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

y_pred = pipeline.predict(X_test)
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"F1: {f1_weighted:.2f}")

F1: 0.65


In [272]:
import joblib
joblib.dump(pipeline, "trained/text2attr.pkl")

['trained/text2attr.pkl']